Merging divisions per season and saving in csv fil

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
def load_merge_data():
    df = pd.DataFrame()
    bool=False
    for subdir, dirs, files in os.walk('.'):
        #print(subdir)
        for file in files:
            #print(file)
            ext = os.path.splitext(file)[-1].lower()
            if ext == ".csv":
                datapath=os.path.join(subdir, file)
                #print(datapath)
                try:
                    data=pd.read_csv(datapath, header=0)
                    print(data.head())
                    #print(data)
                    df=pd.concat([df, data], axis=0, ignore_index=True)
                    bool=True
                    #print(d)
                    #print(datapath)
                except Exception:
                    print("Cannot read files: ",datapath)
        if bool:
            df.to_csv(subdir + '.csv')
        df = pd.DataFrame()
        
        return df
        

In [3]:
#data=pd.read_csv('.\csv files\Season 1993_1994\D1.csv', header=0, index_col=0)
#data.head()
df = load_merge_data()

   Unnamed: 0   id  match_api_id  home_team_api_id  away_team_api_id  \
0         145  540        838599              8635             10001   
1         153  549        838622              9991              8203   
2         155  552        838625             10000              9994   
3         162  563        838646              9984              9991   
4         168  569        838658              9986             10001   

      season        date result_label  away_Defender_rating  \
0  2010/2011  2010-10-24     HOME_WIN             64.500000   
1  2010/2011  2010-10-30     HOME_WIN             64.600000   
2  2010/2011  2010-10-30         DRAW             64.000000   
3  2010/2011  2010-11-07     AWAY_WIN             63.000000   
4  2010/2011  2010-11-13     AWAY_WIN             64.666667   

   home_Defender_rating              ...                home_Midfielder_bmi  \
0             72.666667              ...                          21.744214   
1             65.666667       

#### Scrape Data For Features 


In [4]:
import sqlite3

database = sqlite3.connect('database.sqlite')

#look up table names
all_tables = pd.read_sql("SELECT * FROM sqlite_master;", database)
#print(all_tables)

players = all_tables.get(4)
players = pd.read_sql("SELECT * FROM Player;", database)
matches = pd.read_sql("SELECT * FROM Match;", database)
leagues = pd.read_sql("SELECT * FROM League;", database)
countries = pd.read_sql("SELECT * FROM Country;", database)
teams = pd.read_sql("SELECT * FROM Team;", database)
team_attributes = pd.read_sql("SELECT * FROM Team_Attributes;", database)
player_attributes = pd.read_sql("SELECT * FROM Player_Attributes;",database)

table_list = [players, matches, leagues, countries, teams, team_attributes, player_attributes]
total_list = pd.DataFrame()
#for table in table_list:
#   print '\n',table.info(),table.head()
    
total_list = pd.concat(table_list, axis=1, keys=['players','matches','leagues','countries','teams','team_attributes','player_attributes'])

print teams.head()

   id  team_api_id  team_fifa_api_id     team_long_name team_short_name
0   1         9987             673.0           KRC Genk             GEN
1   2         9993             675.0       Beerschot AC             BAC
2   3        10000           15005.0   SV Zulte-Waregem             ZUL
3   4         9994            2007.0   Sporting Lokeren             LOK
4   5         9984            1750.0  KSV Cercle Brugge             CEB


In [5]:
# total_list.to_csv('mergedlist.csv', encoding='utf-8')

team_barcelona = teams.loc[teams['team_long_name'] == 'FC Barcelona']

print team_barcelona.loc[:'team_api_id']
print team_barcelona.iloc[0]['team_api_id']

teams.to_csv('teams.csv', encoding='utf-8')


        id  team_api_id  team_fifa_api_id team_long_name team_short_name
258  43042         8634             241.0   FC Barcelona             BAR
8634


### Team Rating Scrapping
- Team fifa ratings
- NLP team sentiment

In [6]:
from bs4 import BeautifulSoup
import GetOldTweets
import requests
import urllib, json

print teams.head()

   id  team_api_id  team_fifa_api_id     team_long_name team_short_name
0   1         9987             673.0           KRC Genk             GEN
1   2         9993             675.0       Beerschot AC             BAC
2   3        10000           15005.0   SV Zulte-Waregem             ZUL
3   4         9994            2007.0   Sporting Lokeren             LOK
4   5         9984            1750.0  KSV Cercle Brugge             CEB


In [9]:
team_name = teams.loc[teams['team_api_id'] == 9987]

name = team_name['team_long_name']

# tweet_criteria = GetOldTweets.got.manager.TweetCriteria().setQuerySearch(name.get(0)).setSince("2010-01-01").setMaxTweets(100)

dataframe = pd.DataFrame(columns=dataframeColumns())
webpage = "Global Club Soccer Rankings _ FiveThirtyEight.htm"
#page = urllib.urlopen('https://projects.fivethirtyeight.com/global-club-soccer-rankings/')

with open(webpage) as fp:
    soup = BeautifulSoup(fp, 'lxml')
    
team_names =  soup.find_all('div', attrs={'class':'name'})
    
team_offense_ratings = soup.find_all('td', attrs={'class':'num rating offense drop-6'})

#for b in team_offense_ratings:
    #print b.text
    
team_defense_ratings = soup.find_all('td', attrs={'class':"num rating defense drop-6"})
team_spi_ratings = soup.find_all('td', attrs={'class':"num"})

team_spi_parsed = []
print 'SPI Ratings\n'
for c in range(3, len(team_spi_ratings), 4):
    team_spi_parsed.append(float(team_spi_ratings[c].contents[0]))


team_ranking = soup.find_all('td', attrs={'class':'num ranking'})

for i in range(len(team_names)):
    team_api_id = getTeamId(team_names[i].text, teams)
    dataframe.loc[len(dataframe)] = [team_api_id, team_ranking[i].text, team_names[i].text, team_offense_ratings[i].text, team_defense_ratings[i].text, team_spi_parsed[i]]


team_manchester = teams.loc[teams['team_long_name'] == 'Manchester City']

api_id = teams.iloc[team_manchester.index]['team_api_id']
api = api_id.get_values()

man_loc = dataframe.loc[dataframe['team_long_name'] == 'Man. City']

dataframe.loc[man_loc.index,'team_api_id'] = api[0]

print dataframe.loc[dataframe['team_long_name'] == 'Man. City']['team_api_id']


team_united = teams.loc[teams['team_long_name'] == 'Manchester United']
api_id = teams.iloc[team_united.index]['team_api_id']
api = api_id.get_values()
united_loc = dataframe.loc[dataframe['team_long_name'] == 'Man. United']
dataframe.loc[united_loc.index,'team_api_id'] = api[0]
print dataframe.loc[dataframe['team_long_name'] == 'Man. United']['team_api_id']

dataframe.to_csv('team_rankings.csv', encoding='utf-8')



SPI Ratings

4    8456
Name: team_api_id, dtype: int64
12    10260
Name: team_api_id, dtype: int64


In [10]:
def dataframeColumns():
    return ["team_api_id","team_rank","team_long_name","team_offense_rating","team_defense_rating","team_spi_rating"]

def getTeamId(team_name, teams_csv):
    returned_id = teams_csv.loc[teams_csv['team_short_name'].str.contains(team_name)]
    if (returned_id.empty):
        returned_id = teams_csv.loc[teams_csv['team_long_name'].str.contains(team_name)]
    if (returned_id.empty):
        return 0
    else:
        return returned_id.iloc[0]['team_api_id']
    

### Adding New Features To Large Dataset

In [ ]:
final_csv = pd.read_csv('FINAL_ALL_FEATURES_NO_NAN_final.csv')

new_cols = ['home_team_rating','away_team_rating','home_team_offensive_rating','away_team_offensive_rating','home_team_defensive_rating','away_team_defensive_rating']
for col in new_cols:
    final_csv[col] = 0
i = 0;
first_col_id = final_csv.loc[i,'home_team_api_id']
first_col_matching = dataframe.loc[dataframe['team_api_id'] == first_col_id]
print first_col_matching
print first_col_id

for index in final_csv.index:
    row_home_id = final_csv.loc[index,'home_team_api_id']
    home_col_matching = dataframe.loc[dataframe['team_api_id'] == row_home_id]
    if (home_col_matching.empty):
        final_csv.loc[index,'home_team_rating'] = 0
        final_csv.loc[index, 'home_team_offensive_rating'] = 0
        final_csv.loc[index, 'home_team_defensive_rating'] = 0
    else:
        final_csv[index, 'home_team_rating'] = dataframe.loc[dataframe['team_api_id']==row_home_id, 'team_spi_rating']
        final_csv[index, 'home_team_offensive_rating'] = dataframe.loc[dataframe['team_api_id']==row_home_id, 'team_offense_rating']
        final_csv[index, 'home_team_defensive_rating'] = dataframe.loc[dataframe['team_api_id']==row_home_id, 'team_defense_rating']
    row_away_id = final_csv.loc[index,'away_team_api_id']
    away_col_matching = dataframe.loc[dataframe['team_api_id'] == row_away_id]
    if (away_col_matching.empty):
        final_csv.loc[index,'away_team_rating'] = 0
        final_csv.loc[index, 'away_team_offensive_rating'] = 0
        final_csv.loc[index, 'away_team_defensive_rating'] = 0
    else:
        final_csv[index, 'away_team_rating'] = dataframe.loc[dataframe['team_api_id']==row_away_id, 'team_spi_rating']
        final_csv[index, 'away_team_offensive_rating'] = dataframe.loc[dataframe['team_api_id']==row_away_id, 'team_offense_rating']
        final_csv[index, 'away_team_defensive_rating'] = dataframe.loc[dataframe['team_api_id']==row_away_id, 'team_defense_rating']
        
final_csv.head()


    team_api_id team_rank team_long_name team_offense_rating  \
85         8635        86     Anderlecht                 1.7   

   team_defense_rating  team_spi_rating  
85                 1.1             59.3  
8635


In [ ]:
final_csv['shot_success_rate'] = 0

for index in final_csv.index:
    final_csv[index, 'home_shot_success_rate'] = 
    final_csv[index, 'away_shot_success_rate'] = 